In [1]:

from langchain.storage import InMemoryByteStore
# from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_cohere import CohereEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("Groq_API_KEY")
cohere_api_key = os.getenv("Cohere_API_KEY")

In [2]:
embeddings = CohereEmbeddings(model="embed-english-light-v3.0")

In [2]:
pdf_loader = PyPDFLoader("BTS_Base_Transciever_Station_Structure_I.pdf")
documents = pdf_loader.load()

In [19]:
pdf_loader = TextLoader("C:\\Users\\alizeighamiyan\\Desktop\\genai_bootcamp_daneshkar\\tests\\test3\\readme.txt")
documents = pdf_loader.load()

In [22]:
import ntpath
documents[0].metadata["source"] = ntpath.basename(documents[0].metadata["source"])

In [23]:
documents

[Document(metadata={'source': 'readme.txt'}, page_content='this .ipynb files has ran on google colab and then downloaded to local')]

In [20]:
documents[0].metadata["source"]

'C:\\Users\\alizeighamiyan\\Desktop\\genai_bootcamp_daneshkar\\tests\\test3\\readme.txt'

In [18]:
lens = []
for doc in documents:
    lens.append(len(doc.page_content))


In [5]:
max(lens)

2482

In [6]:
print(documents[7].page_content)

8 
 
 
 
    
 
Fig1: Legacy BTS (cell site). Radio tower and BTS equipment used in a typical cell site 
location. 
 
 
                                 Figure 2. Distributed BTS Architecture


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
text_chunks = text_splitter.split_documents(documents)

In [20]:
len(text_chunks[5].page_content)

1508

In [9]:
len(text_chunks)

90

In [10]:
text_chunks[1]

Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2016-08-27T11:06:04-07:00', 'moddate': '2016-08-27T11:06:04-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'BTS_Base_Transciever_Station_Structure_I.pdf', 'total_pages': 79, 'page': 1, 'page_label': '2'}, page_content='2 \n \n \n \nTABLE OF CONTENTS \n \nCHAPTER 01 INTRODUCTION \n1.1 INTRODUCTION   ………………………………………………………….       07 \nCHAPTER 02 BTS EQUIPMENT \n2.1 Indoor Macro Base Station…………………………………………………...      11 \n2.2 Contents………………………………………………………………………..       11 \n2.3 BTS system overview………………………………………………………….        11 \n2.4 Component of BTS……………………………………………………………        12 \n2.5 BTS Cabinet…………………………………………………………………...         12 \n2.6 BTS Features…………………………………………………………………..       13 \n2.7 BBU Hardware Structure……………………………………………………..       13      \n2.7.1 Logical Structure of BBU……………………………………………………      14 \n2.7.2 Board configuration of BBU…………………………………………………   

In [6]:
db = FAISS.from_documents(text_chunks, embeddings)

In [7]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [8]:
run = itemgetter("question") | retriever
answers = run.invoke({'question': "what is bts?"})

In [9]:
print(answers[4].page_content)

67 
 
 
 
 The BTS, commonly known as 'network tower', requires a large amount of money 
and manpower for maintenance, they added. 
 
 Meanwhile, Robi has sought permission from the telecom regulatory body to hand 
over the maintenance work of its BTS to any associate organisation, according to 
the officials. 
 The regulator--Bangladesh Telecommunication Regulatory Commission (BTRC) 
-- in principle has supported the operators' plea and is scrutinising the existing law 
to find out a way for the job.  
 The commission is considering allowing the operators' move to shift the BTS 
maintenance to associated bodies or third parties as they (operators) want to come 
out of the complexities of tower maintenance," Chairman of the regulatory body 
Major General (retd) Zia Ahmed told BSS. 
 He said the operators have no option to take the initiative of BTS handover 
without the regulator's permission. 
 So, the BTRC is looking for an option as to how the operators can hand over their 
BT

In [10]:
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-specdec")

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant who answers only based on its {context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)

chain = (RunnableParallel(
    {"context": itemgetter("question") | retriever,
     'question': RunnablePassthrough(),
     "chat_history": lambda x: x.get('chat_history')
     }
    ) | prompt | {'out_put': llm.with_config(temprature=0.2) | StrOutputParser()})

In [13]:
response = chain.invoke({'question': "what is bts", 'chat_history':[]})


In [16]:
print(response["out_put"])

BTS stands for Base Transceiver Station. It is a critical component of a mobile phone network that facilitates communication between mobile devices and the network. A BTS is essentially a radio transmitter/receiver that sends and receives signals to and from mobile phones in a specific area, known as a cell. It is usually installed on a tower or mast and is connected to the rest of the network via a backbone network. The BTS is responsible for transmitting and receiving voice and data signals, managing mobile phone connections, and handling handovers between different cells. In other words, it acts as a bridge between the mobile phone and the rest of the network, enabling mobile communication services such as voice calls, text messaging, and internet access.
